In [1]:

import tensorflow as tf
print(tf.__version__)



2.15.0


In [2]:
pip install --upgrade tensorflow

Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install --upgrade tensorflow


Note: you may need to restart the kernel to use updated packages.


In [4]:
import tensorflow as tf
print(tf.__version__)


2.15.0


In [5]:
pip install keras


Note: you may need to restart the kernel to use updated packages.


In [6]:
from keras.layers import Input


In [7]:
from tensorflow.keras.layers import Input
import tensorflow as tf
import pathlib
from tensorflow.keras.initializers import RandomNormal
from tensorflow.keras.layers import Input, Conv2D, Conv2DTranspose, LeakyReLU, Activation, Concatenate, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Model
from numpy import zeros, ones
from numpy.random import randint
from matplotlib import pyplot as plt


In [8]:
import numpy as np
import os
from skimage.transform import resize
from matplotlib import pyplot as plt

# Define file paths
input_dir = r"C:\Users\net pc\Desktop\Thesis Paper 3\Brain tumor dataset\Training\glioma_tumor_inputs"
label_dir = r"C:\Users\net pc\Desktop\Thesis Paper 3\Brain tumor dataset\Training\glioma_tumor_targets"


def load_data(input_dir, label_dir, target_shape):
    input_files = os.listdir(input_dir)
    label_files = os.listdir(label_dir)
    
    # Sort files to ensure corresponding inputs and labels match
    input_files.sort()
    label_files.sort()
    
    trainA = []
    trainB = []
    for i in range(len(input_files)):
        input_img = plt.imread(os.path.join(input_dir, input_files[i]))
        label_img = plt.imread(os.path.join(label_dir, label_files[i]))
        
        # Resize images to target shape if they have different shapes
        if input_img.shape != target_shape:
            input_img = resize(input_img, target_shape)
        if label_img.shape != target_shape:
            label_img = resize(label_img, target_shape)
        
        trainA.append(input_img)
        trainB.append(label_img)
    
    return np.array(trainA), np.array(trainB)



# Define target shape for images (e.g., (256, 256, 3))
target_shape = (256, 256, 3)

# Load data
data = load_data(input_dir, label_dir, target_shape)
print("Data loaded successfully!")


Data loaded successfully!


In [9]:


# Define discriminator
def define_discriminator(image_shape):
    init = RandomNormal(stddev=0.02)
    in_src_image = Input(shape=image_shape, name='input_image')
    in_target_image = Input(shape=image_shape, name='target_image')
    merged = Concatenate()([in_src_image, in_target_image])
    
    # Discriminator architecture
    d = Conv2D(64, (4,4), strides=(2,2), padding='same', kernel_initializer=init)(merged)
    d = LeakyReLU(alpha=0.2)(d)
    d = Conv2D(128, (4,4), strides=(2,2), padding='same', kernel_initializer=init)(d)
    d = BatchNormalization()(d)
    d = LeakyReLU(alpha=0.2)(d)
    d = Conv2D(256, (4,4), strides=(2,2), padding='same', kernel_initializer=init)(d)
    d = BatchNormalization()(d)
    d = LeakyReLU(alpha=0.2)(d)
    d = Conv2D(512, (4,4), strides=(2,2), padding='same', kernel_initializer=init)(d)
    d = BatchNormalization()(d)
    d = LeakyReLU(alpha=0.2)(d)
    d = Conv2D(512, (4,4), padding='same', kernel_initializer=init)(d)
    d = BatchNormalization()(d)
    d = LeakyReLU(alpha=0.2)(d)
    patch_out = Conv2D(1, (4,4), padding='same', kernel_initializer=init)(d)
    patch_out = Activation('sigmoid')(patch_out)
    
    model = Model([in_src_image, in_target_image], patch_out)
    model.compile(loss='binary_crossentropy', optimizer='adam', loss_weights=[0.5])
    return model

In [10]:
def define_generator(image_shape=(256,256,3)):
    init = RandomNormal(stddev=0.02)
    in_image = Input(shape=image_shape)
    
    # Encoder
    e1 = Conv2D(64, (4,4), strides=(2,2), padding='same', kernel_initializer=init)(in_image)
    e2 = Conv2D(128, (4,4), strides=(2,2), padding='same', kernel_initializer=init)(LeakyReLU(alpha=0.2)(e1))
    e2 = BatchNormalization()(e2)
    e3 = Conv2D(256, (4,4), strides=(2,2), padding='same', kernel_initializer=init)(LeakyReLU(alpha=0.2)(e2))
    e3 = BatchNormalization()(e3)
    e4 = Conv2D(512, (4,4), strides=(2,2), padding='same', kernel_initializer=init)(LeakyReLU(alpha=0.2)(e3))
    e4 = BatchNormalization()(e4)
    e5 = Conv2D(512, (4,4), strides=(2,2), padding='same', kernel_initializer=init)(LeakyReLU(alpha=0.2)(e4))
    e5 = BatchNormalization()(e5)
    e6 = Conv2D(512, (4,4), strides=(2,2), padding='same', kernel_initializer=init)(LeakyReLU(alpha=0.2)(e5))
    e6 = BatchNormalization()(e6)
    e7 = Conv2D(512, (4,4), strides=(2,2), padding='same', kernel_initializer=init)(LeakyReLU(alpha=0.2)(e6))
    e7 = BatchNormalization()(e7)
    
    # Bottleneck
    b = Conv2D(512, (4,4), strides=(2,2), padding='same', kernel_initializer=init)(LeakyReLU(alpha=0.2)(e7))
    b = Activation('relu')(b)
    
    # Decoder
    d1 = Conv2DTranspose(512, (4,4), strides=(2,2), padding='same', kernel_initializer=init)(Activation('relu')(b))
    d1 = BatchNormalization()(d1)
    d1 = Dropout(0.5)(d1)
    d1 = Concatenate()([d1, e7])
    d2 = Conv2DTranspose(512, (4,4), strides=(2,2), padding='same', kernel_initializer=init)(Activation('relu')(d1))
    d2 = BatchNormalization()(d2)
    d2 = Dropout(0.5)(d2)
    d2 = Concatenate()([d2, e6])
    d3 = Conv2DTranspose(512, (4,4), strides=(2,2), padding='same', kernel_initializer=init)(Activation('relu')(d2))
    d3 = BatchNormalization()(d3)
    d3 = Concatenate()([d3, e5])
    d4 = Conv2DTranspose(512, (4,4), strides=(2,2), padding='same', kernel_initializer=init)(Activation('relu')(d3))
    d4 = BatchNormalization()(d4)
    d4 = Concatenate()([d4, e4])
    d5 = Conv2DTranspose(256, (4,4), strides=(2,2), padding='same', kernel_initializer=init)(Activation('relu')(d4))
    d5 = BatchNormalization()(d5)
    d5 = Concatenate()([d5, e3])
    d6 = Conv2DTranspose(128, (4,4), strides=(2,2), padding='same', kernel_initializer=init)(Activation('relu')(d5))
    d6 = BatchNormalization()(d6)
    d6 = Concatenate()([d6, e2])
    d7 = Conv2DTranspose(64, (4,4), strides=(2,2), padding='same', kernel_initializer=init)(Activation('relu')(d6))
    d7 = BatchNormalization()(d7)
    d7 = Concatenate()([d7, e1])
    
    # Output
    g = Conv2DTranspose(image_shape[2], (4,4), strides=(2,2), padding='same', kernel_initializer=init)(Activation('tanh')(d7))
    out_image = Activation('tanh')(g)
    
    model = Model(in_image, out_image)
    return model


In [11]:
def define_gan(g_model, d_model, image_shape):
    # Make weights in the discriminator not trainable
    for layer in d_model.layers:
        if not isinstance(layer, BatchNormalization):
            layer.trainable = False

    # Define the source image
    in_src = Input(shape=image_shape)
    
    # Supply the image as input to the generator 
    gen_out = g_model(in_src)
    
    # Supply the input image and generated image as inputs to the discriminator
    dis_out = d_model([in_src, gen_out])
    
    # Combine source image input, generated image, and discriminator output into a model
    model = Model(in_src, [dis_out, gen_out])
    
    return model



In [12]:


def define_combined_model(g_model, d_model, image_shape):
    # Define the source image
    in_src = Input(shape=image_shape)
    
    # Supply the image as input to the generator 
    gen_out = g_model(in_src)
    
    # Supply the input image and generated image as inputs to the discriminator
    dis_out = d_model([in_src, gen_out])
    
    # Combine source image input, generated image, and discriminator output into a model
    model = Model(in_src, [dis_out, gen_out])
    
    # Define GAN model
    gan_model = define_gan(g_model, d_model, image_shape)

    # Compile the GAN model
    opt = Adam(lr=0.0002, beta_1=0.5)
    gan_model.compile(loss=['binary_crossentropy', 'mae'], optimizer=opt, loss_weights=[1, 100])

    return model


In [13]:
# select a batch of random samples, returns images and target
def generate_real_samples(dataset, n_samples, patch_shape):
	# unpack dataset
	trainA, trainB = dataset
	# choose random instances
	ix = randint(0, trainA.shape[0], n_samples)
	# retrieve selected images
	X1, X2 = trainA[ix], trainB[ix]
	# generate 'real' class labels (1)
	y = ones((n_samples, patch_shape, patch_shape, 1))
	return [X1, X2], y

In [14]:
# generate a batch of images, returns images and targets
def generate_fake_samples(g_model, samples, patch_shape):
	# generate fake instance
	X = g_model.predict(samples)
	# create 'fake' class labels (0)
	y = zeros((len(X), patch_shape, patch_shape, 1))
	return X, y


In [15]:

# generate samples and save as a plot and save the model
#GAN models do not converge, we just want to find a good balance between
#the generator and the discriminator. Therefore, it makes sense to periodically
#save the generator model and check how good the generated image looks. 


import os
from keras.preprocessing.image import save_img

def summarize_performance(step, g_model, dataset, n_samples=3):
    print("Saving images...")
    # select a sample of input images
    [X_realA, X_realB], _ = generate_real_samples(dataset, n_samples, 1)
    # generate a batch of fake samples
    X_fakeB, _ = generate_fake_samples(g_model, X_realA, 1)
    # scale all pixels from [-1,1] to [0,1]
    X_realA = (X_realA + 1) / 2.0
    X_realB = (X_realB + 1) / 2.0
    X_fakeB = (X_fakeB + 1) / 2.0
    
    output_path = "C:\Users\net pc\Desktop\Thesis Paper 3\Brain tumor dataset\glioma_tumor_generated_Pix2Pix"
    
    for i in range(n_samples):
        # save real source image as JPEG
        real_source_image = X_realA[i]
        real_source_filename = os.path.join(output_path, f'real_source_{step}_{i + 1}.jpeg')
        save_img(real_source_filename, real_source_image)

        # save generated target image as JPEG
        generated_target_image = X_fakeB[i]
        generated_target_filename = os.path.join(output_path, f'generated_target_{step}_{i + 1}.jpeg')
        save_img(generated_target_filename, generated_target_image)

        # save real target image as JPEG
        real_target_image = X_realB[i]
        real_target_filename = os.path.join(output_path, f'real_target_{step}_{i + 1}.jpeg')
        save_img(real_target_filename, real_target_image)

    # save the generator model
    model_filename = os.path.join(output_path, f'model_{step}.h5')
    g_model.save(model_filename)

    print(f"Images and model saved at {output_path}")



In [16]:

# train pix2pix models
def train(d_model, g_model, gan_model, dataset, n_epochs=100, n_batch=1):
	# determine the output square shape of the discriminator
	n_patch = d_model.output_shape[1]
	# unpack dataset
	trainA, trainB = dataset
	# calculate the number of batches per training epoch
	bat_per_epo = int(len(trainA) / n_batch)
	# calculate the number of training iterations
	n_steps = bat_per_epo * n_epochs
	# manually enumerate epochs
	for i in range(n_steps):
		# select a batch of real samples
		[X_realA, X_realB], y_real = generate_real_samples(dataset, n_batch, n_patch)
		# generate a batch of fake samples
		X_fakeB, y_fake = generate_fake_samples(g_model, X_realA, n_patch)
		# update discriminator for real samples
		d_loss1 = d_model.train_on_batch([X_realA, X_realB], y_real)
		# update discriminator for generated samples
		d_loss2 = d_model.train_on_batch([X_realA, X_fakeB], y_fake)
		# update the generator
		g_loss, _, _ = gan_model.train_on_batch(X_realA, [y_real, X_realB])
		# summarize performance
		print('>%d, d1[%.3f] d2[%.3f] g[%.3f]' % (i+1, d_loss1, d_loss2, g_loss))
		# summarize model performance
		if (i+1) % (bat_per_epo * 10) == 0:
			summarize_performance(i, g_model, dataset)


In [17]:
# Define discriminator model
d_model = define_discriminator(target_shape)

# Define generator model
g_model = define_generator(target_shape)

# Define GAN model
gan_model = define_gan(g_model, d_model, target_shape)

# Define combined model
combined_model = define_combined_model(g_model, d_model, target_shape)



C:\Users\Public\Efendim\Lib\site-packages\keras\src\initializers\initializers.py:120: UserWarning: The initializer RandomNormal is unseeded and being called multiple times, which will return identical values each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initializer instance more than once.
  warnings.warn(


In [18]:
# Define GAN model
gan_model = define_gan(g_model, d_model, target_shape)

# Compile the GAN model
opt = Adam(lr=0.0002, beta_1=0.5)
gan_model.compile(loss=['binary_crossentropy', 'mae'], optimizer=opt, loss_weights=[1, 100])

# Train Pix2Pix models using the GAN model
train(d_model, g_model, gan_model, data, n_epochs=10000, n_batch=1)


1/1 [==============================] - 0s 308ms/step



>1, d1[0.348] d2[2.469] g[32.757]
1/1 [==============================] - 0s 71ms/step
>2, d1[1.510] d2[2.651] g[30.252]
1/1 [==============================] - 0s 73ms/step
>3, d1[0.399] d2[1.146] g[40.300]
1/1 [==============================] - 0s 74ms/step
>4, d1[0.415] d2[0.565] g[15.151]
1/1 [==============================] - 0s 78ms/step
>5, d1[0.521] d2[0.292] g[25.171]
1/1 [==============================] - 0s 72ms/step
>6, d1[0.605] d2[0.248] g[41.269]
1/1 [==============================] - 0s 86ms/step
>7, d1[0.591] d2[0.269] g[12.913]
1/1 [==============================] - 0s 71ms/step
>8, d1[0.427] d2[0.370] g[27.045]
1/1 [==============================] - 0s 76ms/step
>9, d1[0.361] d2[0.418] g[15.231]
1/1 [==============================] - 0s 95ms/step
>10, d1[0.341] d2[0.408] g[26.843]
1/1 [==============================] - 0s 73ms/step
>11, d1[0.340] d2[0.379] g[19.730]
1/1 [==============================] - 0s 79ms/step
>12, d1[0.381] d2[0.348] g[13.207]
1/1 [============

1/1 [==============================] - 0s 72ms/step
>96, d1[0.017] d2[0.203] g[14.051]
1/1 [==============================] - 0s 73ms/step
>97, d1[0.035] d2[0.044] g[18.416]
1/1 [==============================] - 0s 72ms/step
>98, d1[0.489] d2[0.041] g[15.441]
1/1 [==============================] - 0s 71ms/step
>99, d1[0.130] d2[0.094] g[13.999]
1/1 [==============================] - 0s 77ms/step
>100, d1[0.054] d2[0.072] g[15.757]
1/1 [==============================] - 0s 72ms/step
>101, d1[0.027] d2[0.113] g[26.468]
1/1 [==============================] - 0s 83ms/step
>102, d1[0.030] d2[0.139] g[13.135]
1/1 [==============================] - 0s 71ms/step
>103, d1[0.036] d2[0.072] g[11.457]
1/1 [==============================] - 0s 74ms/step
>104, d1[0.035] d2[0.059] g[14.317]
1/1 [==============================] - 0s 72ms/step
>105, d1[0.065] d2[0.030] g[10.910]
1/1 [==============================] - 0s 73ms/step
>106, d1[0.016] d2[0.024] g[14.022]
1/1 [==============================]

1/1 [==============================] - 0s 71ms/step
>189, d1[0.493] d2[0.034] g[23.093]
1/1 [==============================] - 0s 78ms/step
>190, d1[0.009] d2[0.168] g[14.815]
1/1 [==============================] - 0s 74ms/step
>191, d1[0.004] d2[0.120] g[9.185]
1/1 [==============================] - 0s 81ms/step
>192, d1[0.044] d2[0.035] g[23.264]
1/1 [==============================] - 0s 74ms/step
>193, d1[0.009] d2[0.056] g[9.229]
1/1 [==============================] - 0s 73ms/step
>194, d1[0.003] d2[0.027] g[13.769]
1/1 [==============================] - 0s 86ms/step
>195, d1[0.006] d2[0.017] g[7.421]
1/1 [==============================] - 0s 73ms/step
>196, d1[0.003] d2[0.013] g[12.255]
1/1 [==============================] - 0s 74ms/step
>197, d1[0.002] d2[0.011] g[10.751]
1/1 [==============================] - 0s 93ms/step
>198, d1[0.005] d2[0.014] g[18.251]
1/1 [==============================] - 0s 70ms/step
>199, d1[0.004] d2[0.018] g[15.973]
1/1 [==============================

1/1 [==============================] - 0s 73ms/step
>282, d1[0.001] d2[0.003] g[22.927]
1/1 [==============================] - 0s 74ms/step
>283, d1[0.001] d2[0.003] g[10.738]
1/1 [==============================] - 0s 71ms/step
>284, d1[0.001] d2[0.002] g[12.227]
1/1 [==============================] - 0s 72ms/step
>285, d1[0.002] d2[0.002] g[8.917]
1/1 [==============================] - 0s 88ms/step
>286, d1[0.001] d2[0.002] g[17.417]
1/1 [==============================] - 0s 72ms/step
>287, d1[0.001] d2[0.004] g[11.006]
1/1 [==============================] - 0s 68ms/step
>288, d1[0.001] d2[0.002] g[9.596]
1/1 [==============================] - 0s 76ms/step
>289, d1[0.001] d2[0.003] g[13.297]
1/1 [==============================] - 0s 71ms/step
>290, d1[0.001] d2[0.003] g[11.101]
1/1 [==============================] - 0s 75ms/step
>291, d1[0.002] d2[0.002] g[11.084]
1/1 [==============================] - 0s 77ms/step
>292, d1[0.001] d2[0.002] g[17.288]
1/1 [=============================

...

1/1 [==============================] - 0s 77ms/step
>747, d1[0.108] d2[0.122] g[13.945]
1/1 [==============================] - 0s 73ms/step
>748, d1[0.039] d2[0.069] g[19.345]
1/1 [==============================] - 0s 72ms/step
>749, d1[0.177] d2[0.076] g[15.653]
1/1 [==============================] - 0s 93ms/step
>750, d1[0.220] d2[0.086] g[13.253]
1/1 [==============================] - 0s 74ms/step
>751, d1[0.113] d2[0.110] g[16.699]
1/1 [==============================] - 0s 74ms/step
>752, d1[0.034] d2[0.131] g[18.430]
1/1 [==============================] - 0s 72ms/step
>753, d1[0.054] d2[0.189] g[14.707]
1/1 [==============================] - 0s 72ms/step
>754, d1[0.392] d2[0.248] g[11.868]
1/1 [==============================] - 0s 88ms/step
>755, d1[0.028] d2[0.066] g[16.971]
1/1 [==============================] - 0s 69ms/step
>756, d1[0.153] d2[0.182] g[14.481]
1/1 [==============================] - 0s 76ms/step
>757, d1[0.045] d2[0.124] g[14.315]
1/1 [===========================

1/1 [==============================] - 0s 85ms/step
>840, d1[0.278] d2[0.104] g[11.674]
1/1 [==============================] - 0s 73ms/step
>841, d1[0.032] d2[0.147] g[23.449]
1/1 [==============================] - 0s 77ms/step
>842, d1[0.171] d2[0.193] g[10.803]
1/1 [==============================] - 0s 83ms/step
>843, d1[0.041] d2[0.167] g[13.732]
1/1 [==============================] - 0s 68ms/step
>844, d1[0.038] d2[0.122] g[20.262]
1/1 [==============================] - 0s 74ms/step
>845, d1[0.031] d2[0.166] g[28.728]
1/1 [==============================] - 0s 74ms/step
>846, d1[0.238] d2[0.116] g[10.933]
1/1 [==============================] - 0s 70ms/step
>847, d1[0.251] d2[0.111] g[20.750]
1/1 [==============================] - 0s 73ms/step
>848, d1[0.040] d2[0.102] g[3757.338]
1/1 [==============================] - 0s 79ms/step
>849, d1[0.039] d2[0.101] g[16.628]
1/1 [==============================] - 0s 74ms/step
>850, d1[0.035] d2[0.116] g[15.932]
1/1 [=========================

1/1 [==============================] - 0s 72ms/step
>933, d1[0.081] d2[0.300] g[12.985]
1/1 [==============================] - 0s 71ms/step
>934, d1[0.254] d2[0.246] g[9.908]
1/1 [==============================] - 0s 97ms/step
>935, d1[0.090] d2[0.234] g[14.947]
1/1 [==============================] - 0s 72ms/step
>936, d1[0.146] d2[0.240] g[18.976]
1/1 [==============================] - 0s 83ms/step
>937, d1[0.022] d2[0.078] g[17.787]
1/1 [==============================] - 0s 84ms/step
>938, d1[0.024] d2[0.047] g[22.472]
1/1 [==============================] - 0s 76ms/step
>939, d1[0.529] d2[0.044] g[15.190]
1/1 [==============================] - 0s 76ms/step
>940, d1[0.250] d2[0.116] g[11.991]
1/1 [==============================] - 0s 70ms/step
>941, d1[0.461] d2[0.487] g[16.638]
1/1 [==============================] - 0s 73ms/step
>942, d1[0.014] d2[0.444] g[20.119]
1/1 [==============================] - 0s 82ms/step
>943, d1[0.005] d2[0.646] g[16.193]
1/1 [============================

C:\Users\Public\Efendim\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


Images and model saved at C:\Users\net pc\Desktop\Thesis Paper 3\generated_Pix2Pix
